In [4]:
# pinecon client와 vector를 만들기 위한 sentence-transformer를 다운로드 한다
# !pip install -qU pinecone-client==2.2.1 sentence-transformers==2.2.2

In [2]:
# !pip uninstall torch torchvision torchaudio

^C


In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.0.1+cpu
False


In [1]:
import os
import pinecone
# api key를 다른 파일에 적어둠
import pinecon_api_key as pk

# get api key from app.pinecone.io
PINECONE_API_KEY = pk.api_key()
# find your environment next to the api key in pinecone console
PINECONE_ENV = 'northamerica-northeast1-gcp'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

c:\Users\s\anaconda3\envs\pinecone\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import torch
torch.cuda.is_available()

False

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

You are using cpu. This is much slower than using a CUDA-enabled GPU. If on Colab you can change this by clicking Runtime > Change runtime type > GPU.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
# Create a sentence embedding for test
query = 'Tell me about Android OS.'

xq = model.encode(query)
xq.shape

(384,)

In [ ]:
# Now we need to create Pinecone 'index'. It may take about 2~3 minutes to be done.
# This is because Pinecone launches a pod in order to create an index.
index_name = 'semantic-search'

# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine'
    )

# now connect to the index
index = pinecone.GRPCIndex(index_name)

In [ ]:
# Prepare data to use. These are generated using ChatGPT.
data = [
    "Android OS has established itself as a leading mobile operating system, offering a versatile and user-friendly platform. With its customizable interface, extensive app ecosystem, and seamless integration with Google services, Android provides a delightful experience for millions of users worldwide. From sleek smartphones to powerful tablets and smartwatches, Android powers a wide range of devices, catering to diverse needs and preferences. Its open-source nature fosters innovation and empowers developers to create innovative apps and features, making Android an ever-evolving and dynamic ecosystem. With regular updates and improvements, Android continues to shape the future of mobile technology.",
    "Android OS prioritizes security and privacy, offering robust features to protect user data and ensure a safe digital environment. With built-in security measures like app sandboxing, encryption, and regular security updates, Android guards against malware and unauthorized access. Users have granular control over permissions, allowing them to manage app access to sensitive information. Moreover, Android's Play Protect system scans apps for potential threats, providing an additional layer of security. By adhering to privacy standards and empowering users with privacy controls, Android remains committed to safeguarding personal data and maintaining user trust.",
    "Android OS fosters an ecosystem of possibilities, enabling users to personalize their devices and explore a vast array of apps, services, and accessories. With a wide range of customization options, users can tailor their Android experience to suit their unique preferences, from custom launchers and themes to widgets and advanced settings. The Google Play Store offers a vast library of apps, games, and digital content, empowering users with diverse choices. Additionally, Android's compatibility with a range of devices and accessories, such as smart home devices and wearables, expands its potential and enhances the connected experience. Android's versatility fuels endless innovation and opens up a world of opportunities for users worldwide.",
    "Android app development empowers developers to unleash their creativity and bring innovative ideas to life. With a vast user base and a thriving ecosystem, Android provides an immense opportunity to reach a global audience. The Android development framework offers a wide range of tools, libraries, and APIs, making it easier to build feature-rich and interactive applications. From building intuitive user interfaces to leveraging advanced functionalities like location services and device sensors, Android app development allows developers to create powerful apps that cater to diverse user needs. With continuous updates and community support, Android app development remains at the forefront of innovation.",
    "Android app development focuses on creating engaging user experiences that captivate and delight users. The Android platform provides developers with various design principles and guidelines, such as Material Design, to create visually appealing and intuitive interfaces. With access to a wide range of UI components and animations, developers can craft seamless and interactive experiences. Moreover, Android's support for multi-device compatibility ensures that apps can adapt and perform optimally across different screen sizes and resolutions. By leveraging the capabilities of Android app development, developers can create immersive and user-centric applications that leave a lasting impression.",
    "Android app development thrives within a vibrant ecosystem that empowers developers with extensive resources and support. From comprehensive documentation and tutorials to a vast community of developers, Android offers a wealth of knowledge and collaboration opportunities. The Google Play Store serves as a global distribution platform, allowing developers to showcase their apps to millions of users. Additionally, the integration of Google services and APIs provides developers with powerful tools to enhance their apps with features like maps, push notifications, and analytics. Android's ecosystem fosters growth, learning, and collaboration, enabling developers to excel in creating innovative and impactful applications.",
    "iPhone app development is synonymous with crafting exquisite experiences for iOS users. Apple's iOS platform offers a refined and intuitive user interface that emphasizes elegance and simplicity. Developers have access to a powerful set of development tools, including the Xcode IDE and Swift programming language, enabling them to create high-quality and polished applications. The App Store provides a curated marketplace for iPhone apps, ensuring that users can discover and download apps with ease. Apple's strict app review process ensures that apps meet quality and security standards, instilling trust in users. With a strong focus on user experience, performance optimization, and adherence to design guidelines, iPhone app development allows developers to create visually stunning, responsive, and user-centric applications for Apple's dedicated user base.",
    "iPhone app development unlocks the power of iOS and offers seamless integration with the broader Apple ecosystem. Developers can leverage a wide range of Apple's proprietary technologies and frameworks, such as CoreML for machine learning, ARKit for augmented reality, and HealthKit for health and fitness applications. Integration with Apple services like iCloud enables seamless data synchronization across devices, while Apple Pay facilitates secure and convenient in-app purchases. Furthermore, features like SiriKit allow developers to incorporate voice control and interactions into their apps. By embracing the Apple ecosystem, iPhone app development enables developers to create innovative and interconnected experiences that enhance user productivity, entertainment, and well-being.",
    "Privacy and security are core tenets of iPhone app development. Apple prioritizes user data protection by implementing robust security measures and privacy features. App developers must adhere to strict guidelines regarding data collection, storage, and sharing. Apple's App Store review process scrutinizes apps for potential privacy violations, ensuring that users' personal information remains secure. Technologies like Face ID and Touch ID provide convenient and secure authentication options, bolstering user trust. Additionally, App Tracking Transparency empowers users to control which apps can track their activity for personalized advertising. By embracing privacy and security, iPhone app development aligns with Apple's commitment to protecting user data and upholding the trust of millions of iOS users."
]

In [ ]:
# Create embeddings and upsert to Piencone index
ids = [str(x) for x in range(0, len(data))]
metadatas = [{'text': text} for text in data]
xc = model.encode(data)
records = zip(ids, xc, metadatas)
index.upsert(vectors=records)

# check number of records in the index
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 9}},
 'total_vector_count': 9}

In [ ]:
def make_query(query):
    # create the query vector
    xq = model.encode(query).tolist()

    # now query
    xc = index.query(xq, top_k=2, include_metadata=True)
    result = [{'score': match['score'], 'text': match['metadata']['text']} for match in xc['matches']]
    return result

In [ ]:
make_query("What is Android OS?")

[{'score': 0.6553541,
  'text': 'Android OS has established itself as a leading mobile operating system, offering a versatile and user-friendly platform. With its customizable interface, extensive app ecosystem, and seamless integration with Google services, Android provides a delightful experience for millions of users worldwide. From sleek smartphones to powerful tablets and smartwatches, Android powers a wide range of devices, catering to diverse needs and preferences. Its open-source nature fosters innovation and empowers developers to create innovative apps and features, making Android an ever-evolving and dynamic ecosystem. With regular updates and improvements, Android continues to shape the future of mobile technology.'},
 {'score': 0.5829659,
  'text': "Android OS fosters an ecosystem of possibilities, enabling users to personalize their devices and explore a vast array of apps, services, and accessories. With a wide range of customization options, users can tailor their Andro

In [ ]:
make_query("I want to know about App development.")

[{'score': 0.57881624,
  'text': 'Android app development empowers developers to unleash their creativity and bring innovative ideas to life. With a vast user base and a thriving ecosystem, Android provides an immense opportunity to reach a global audience. The Android development framework offers a wide range of tools, libraries, and APIs, making it easier to build feature-rich and interactive applications. From building intuitive user interfaces to leveraging advanced functionalities like location services and device sensors, Android app development allows developers to create powerful apps that cater to diverse user needs. With continuous updates and community support, Android app development remains at the forefront of innovation.'},
 {'score': 0.57174355,
  'text': "iPhone app development is synonymous with crafting exquisite experiences for iOS users. Apple's iOS platform offers a refined and intuitive user interface that emphasizes elegance and simplicity. Developers have access

In [ ]:
#
make_query("아이폰에 대해 알고싶어.")

[{'score': 0.19363675,
  'text': 'Android OS has established itself as a leading mobile operating system, offering a versatile and user-friendly platform. With its customizable interface, extensive app ecosystem, and seamless integration with Google services, Android provides a delightful experience for millions of users worldwide. From sleek smartphones to powerful tablets and smartwatches, Android powers a wide range of devices, catering to diverse needs and preferences. Its open-source nature fosters innovation and empowers developers to create innovative apps and features, making Android an ever-evolving and dynamic ecosystem. With regular updates and improvements, Android continues to shape the future of mobile technology.'},
 {'score': 0.17957823,
  'text': "Android app development focuses on creating engaging user experiences that captivate and delight users. The Android platform provides developers with various design principles and guidelines, such as Material Design, to creat